In [1]:
!pip install mysql-connector-python sqlalchemy pymysql  


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas requests beautifulsoup4 yfinance tqdm



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# import thư viện
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import pandas as pd
from tqdm import tqdm
import mysql.connector

In [4]:
# Hàm 1: Crawl dữ liệu từ JPX và tiền xử lý

url = 'https://www.jpx.co.jp/english/equities/products/etfs/issues/01.html'
crawl = requests.get(url)
soup = BeautifulSoup(crawl.text,'html.parser')

def get_all_codes(soup):
    table = soup.find("table")
    codes = []

    if table:
        for tr in table.find_all("tr")[1:]:
            cols = tr.find_all("td")
            if len(cols) >= 3:  
                code = cols[2].get_text(strip=True)
                codes.append(code)

    return codes

all_code_order = get_all_codes(soup)      

category_links = []

for ul in soup.select("ul.input-btn-list.column6-boxIn"):
    for a in ul.select("a"):
        href = a.get("href", "").strip()
        text = a.get_text(strip=True)

        # BỎ url "All"
        if href == "/english/equities/products/etfs/issues/01.html":
            continue

        # lấy 11 category có dạng 01-xx
        if "/english/equities/products/etfs/issues/01-" in href:
            full_url = "https://www.jpx.co.jp" + href
            category_links.append((text, full_url))

print(category_links)
print(len(category_links))  

# crawl 7 cột data

def parse_etf_table(soup):
    table = soup.find("table")
    columns = ["Listing Date", "Index", "Code", "Fund Name",
               "Management Company", "Trading Unit", "Trust Fee"]
    
    if table is None:
        return pd.DataFrame(columns=columns)
    
    rows = []
    for tr in table.find_all("tr")[1:]:
        cols = tr.find_all("td")
        if len(cols) < 7:
            continue
        row = [cols[i].get_text(" ", strip=True) for i in range(7)]
        rows.append(row)
    
    return pd.DataFrame(rows, columns=columns)

def parse_etf_table(soup):
    table = soup.find("table")
    columns = ["Listing Date", "Index", "Code", "Fund Name",
               "Management Company", "Trading Unit", "Trust Fee", "Tags"]
    
    if table is None:
        return pd.DataFrame(columns=columns)
    
    rows = []
    for tr in table.find_all("tr")[1:]:
        cols = tr.find_all("td")
        if len(cols) < 7:
            continue
        
        # ---- Lấy FUND NAME sạch ----
        fund_td = cols[3]
        fund_name = fund_td.find(text=True, recursive=False).strip()

        # ---- Lấy các TAG ----
        tags = [a.get_text(strip=True) for a in fund_td.find_all("a")]

        row = [
            cols[0].get_text(" ", strip=True),
            cols[1].get_text(" ", strip=True),
            cols[2].get_text(" ", strip=True),
            fund_name,
            cols[4].get_text(" ", strip=True),
            cols[5].get_text(" ", strip=True),
            cols[6].get_text(" ", strip=True),
            ", ".join(tags)
        ]
        rows.append(row)
    
    return pd.DataFrame(rows, columns=columns)

# Crawl 11 etf_type

final_df = pd.DataFrame()

for cat_name, cat_url in category_links:
    print("Crawling:", cat_name, cat_url)
    html = requests.get(cat_url).text
    soup_cat = BeautifulSoup(html, "html.parser")
    
    df_cat = parse_etf_table(soup_cat)
    if df_cat.empty:
        continue  # bỏ table rỗng
    
    df_cat["Category"] = cat_name
    final_df = pd.concat([final_df, df_cat], ignore_index=True)

# sắp xếp lại thứ tự theo url "All"

final_df["Code"] = pd.Categorical(final_df["Code"], categories=all_code_order, ordered=True)
final_df = final_df.sort_values("Code").reset_index(drop=True)


    


[('Japanese Equity(Market)', 'https://www.jpx.co.jp/english/equities/products/etfs/issues/01-01.html'), ('Japanese Equity(Size)', 'https://www.jpx.co.jp/english/equities/products/etfs/issues/01-02.html'), ('Japanese Equity(Sector)', 'https://www.jpx.co.jp/english/equities/products/etfs/issues/01-03.html'), ('Japanese Equity(Theme)', 'https://www.jpx.co.jp/english/equities/products/etfs/issues/01-04.html'), ('Foreign Equity', 'https://www.jpx.co.jp/english/equities/products/etfs/issues/01-08.html'), ('Bond', 'https://www.jpx.co.jp/english/equities/products/etfs/issues/01-09.html'), ('REIT', 'https://www.jpx.co.jp/english/equities/products/etfs/issues/01-07.html'), ('Commodity/ Commodity Index', 'https://www.jpx.co.jp/english/equities/products/etfs/issues/01-10.html'), ('Commodity/ Commodity Index(ETC)', 'https://www.jpx.co.jp/english/equities/products/etfs/issues/01-11.html'), ('Enhanced', 'https://www.jpx.co.jp/english/equities/products/etfs/issues/01-05.html'), ('Balanced', 'https://w

C:\Users\Admin\AppData\Local\Temp\ipykernel_8544\1515370340.py:77: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  fund_name = fund_td.find(text=True, recursive=False).strip()


Crawling: Japanese Equity(Size) https://www.jpx.co.jp/english/equities/products/etfs/issues/01-02.html


C:\Users\Admin\AppData\Local\Temp\ipykernel_8544\1515370340.py:77: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  fund_name = fund_td.find(text=True, recursive=False).strip()


Crawling: Japanese Equity(Sector) https://www.jpx.co.jp/english/equities/products/etfs/issues/01-03.html


...

Crawling: Commodity/ Commodity Index(ETC) https://www.jpx.co.jp/english/equities/products/etfs/issues/01-11.html


C:\Users\Admin\AppData\Local\Temp\ipykernel_8544\1515370340.py:77: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  fund_name = fund_td.find(text=True, recursive=False).strip()


Crawling: Enhanced https://www.jpx.co.jp/english/equities/products/etfs/issues/01-05.html


C:\Users\Admin\AppData\Local\Temp\ipykernel_8544\1515370340.py:77: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  fund_name = fund_td.find(text=True, recursive=False).strip()


Crawling: Balanced https://www.jpx.co.jp/english/equities/products/etfs/issues/01-12.html


In [5]:
# Hàm 2: lấy yield
def get_yield(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        if "dividendYield" in info and info["dividendYield"] is not None:
            return info["dividendYield"]
        else:
            return None
    except:
        return None

# lấy yield
yields = []
for code in tqdm(final_df["Code"]):
    ticker = f"{code}.T"
    yields.append(get_yield(ticker))

final_df["yield"] = yields

100%|██████████████████████████████████████████████████████████████████████████████| 357/357 [01:48<00:00,  3.29it/s]


In [6]:
# Hàm 3: xử lí cuối cùng

final_df = final_df.rename(columns={
    'Listing Date' : 'listing_date',
    'Index' : 'index_name',
    'Code' : 'etf_code',
    'Fund Name' : 'etfn_name',
    'Management Company' : 'management_company',
    'Trading Unit' : 'trading_unit',
    'Trust Fee' : 'trust_fee',
    'Category' : 'etf_type',
    'yield' : 'distribution_yield'

})

final_df = final_df[['listing_date','etf_code','etfn_name','index_name','trading_unit','management_company','trust_fee','distribution_yield','etf_type']]

final_df = final_df.reset_index()
final_df['index'] = final_df['index'] + 1
final_df = final_df.rename(columns={'index':'id'})

final_df['listing_date'] = pd.to_datetime(final_df['listing_date'], errors='coerce', format='mixed')
final_df['etf_code'] = final_df['etf_code'].astype(dtype=object)

In [7]:
# --- Hàm 4: UPSERT MYSQL
def upsert_to_mysql(df: pd.DataFrame, table_name: str = 'etf_info'):
    """
    Thực hiện kết nối và UPSERT (Update or Insert) dữ liệu từ DataFrame vào MySQL.
    Sử dụng 'etf_code' làm khóa DUY NHẤT để kiểm tra trùng lặp.
    """
    try:
        # 1. Kết nối MySQL (Sử dụng thông tin của bạn)
        cnx = mysql.connector.connect(
            user='root',
            password='Phunghainam2004!',
            host='127.0.0.1',
            database='train'
        )
        cursor = cnx.cursor()

        # 2. Định nghĩa các cột
        # Bỏ cột 'id' nếu nó là AUTO_INCREMENT. Ở đây, ta dùng tất cả cột trừ 'id' để update.
        cols_to_use = [
            'listing_date', 'etf_code', 'etfn_name', 'index_name', 
            'trading_unit', 'management_company', 'trust_fee', 
            'distribution_yield', 'etf_type'
        ]
        
        # 3. Tạo cú pháp SQL cho UPSERT
        col_str = ", ".join([f"`{c}`" for c in cols_to_use])
        placeholder_str = ", ".join(["%s"] * len(cols_to_use))
        
        # Tạo chuỗi cho phần ON DUPLICATE KEY UPDATE
        # Các cột sẽ được cập nhật nếu 'etf_code' đã tồn tại
        update_str = ", ".join([f"`{c}` = VALUES(`{c}`)" for c in cols_to_use])

        sql = f"""
        INSERT INTO `{table_name}` ({col_str})
        VALUES ({placeholder_str})
        ON DUPLICATE KEY UPDATE
        {update_str}
        """

        # 4. Chuẩn bị dữ liệu và thực thi
        df_to_insert = df[cols_to_use]
        data_to_insert = [
            tuple(None if pd.isna(x) else x for x in row) 
            for row in df_to_insert.itertuples(index=False, name=None)
        ]
        
        # Sử dụng executemany để chèn nhiều dòng cùng lúc (tối ưu hơn loop)
        cursor.executemany(sql, data_to_insert)
        
        # 5. Commit và Đóng kết nối
        cnx.commit()
        print(f"✅ Đã UPSERT {cursor.rowcount} dòng vào bảng `{table_name}` thành công!")

    except mysql.connector.Error as err:
        print(f"❌ Lỗi MySQL: {err}")
    
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'cnx' in locals() and cnx and cnx.is_connected():
            cnx.close()


# --- THỰC THI HÀM 5 (Chạy cell này trong Notebook) ---

# Đảm bảo cột 'id' được loại bỏ nếu bạn không muốn insert giá trị cho nó (AUTO_INCREMENT)
# Nếu bạn muốn dùng cột 'id' đã tạo để làm khóa chính, bạn cần thêm nó vào `cols_to_use`.
# Ở đây ta chỉ dùng các cột có dữ liệu crawl được.
df_for_upsert = final_df.drop(columns=['id'])

upsert_to_mysql(df_for_upsert)

✅ Đã UPSERT 0 dòng vào bảng `etf_info` thành công!
